## Part 1
### PLAN
- create a tree node
- build the tree
  - keep track of path by storing a node list
  - nodes will contain: name, children, size
  - fill nodes with the immediate size
- traverse the tree
  - use DFS to aggregate to the total_size through the worst case

### Discoveries
- Multiple dirs can share the same name
- Sooooo thats why you use slashes in OS directory trees...
  - makes each path unique and is easily parsed



In [29]:
class TreeNode:
    def __init__(self, val):
        self.val = val
        self.dir_name = val.split(" ")[-1]
        self.size = 0
        self.children = {}

with open('input.txt') as f:
    lines = f.readlines()

def build_tree(lines):
    current_path = []
    current_path_str = ""
    nodes = {}
    
    for line in lines:
        # will ignore "$ ls"
        if line.startswith("$"):
            if line.startswith("$ cd"):
                # print(current_path_str)
                if line.endswith("..\n"):
                    dir_node = current_path.pop()
                    current_path_str = current_path_str[:-(len(dir_node.dir_name) + 1)]

                else:
                    dir_name = line[5:-1]
                    current_path_str += " "+dir_name
                    if (current_path_str not in nodes):
                        nodes[current_path_str] = TreeNode(current_path_str)
                    current_path.append(nodes[current_path_str])
        # files and dirs
        else:
            if line.startswith("dir"):
                dir_name = line[4:-1]
                temp_dir_str = current_path_str + " " + dir_name
                if (temp_dir_str not in nodes):
                    nodes[temp_dir_str] = TreeNode(temp_dir_str)
                current_path[-1].children[temp_dir_str] = nodes[temp_dir_str]
            else:
                file_size = int(line.split(" ")[0])
                current_path[-1].size += file_size
    return nodes

def dfs(node, visited=set()):
    # print("PRE: ", node.val, node.size)
    for child in node.children.values():
        # # to avoid cycles
        # if child not in visited:
            # visited.add(child)
        node.size += dfs(child, visited)
    # print("POST: ", node.val, node.size)
    return node.size

def conditioned_result(nodes):
    sum_sizes = 0
    for node in nodes.values():
        if node.size < 100000:
            sum_sizes += node.size
    return sum_sizes

def pipeline1(lines):
    nodes = build_tree(lines)
    # for node in nodes.values():
    #     print(node.val, node.size)
    root = nodes[" /"]
    dfs(root)
    return conditioned_result(nodes)


print(pipeline1(lines))
# NOT: 1167625, 1440660 


1428881


## Part 2
- calculate space needed to be freed
- iterate through nodes and find the one whose size is the smallest amount more than the space needed to be freed
  - could use a binary search tree for optimization (but after spending 2 hours on part 1, I'm not doing that)

In [33]:
with open('input.txt') as f:
    lines = f.readlines()

def conditioned_result_part2(nodes):
    root = nodes[" /"]
    disk_space = 70000000
    free_space_required = 30000000
    free_space = disk_space - root.size
    amt_to_free = free_space_required - free_space
    
    top_candidate = None
    for node in nodes.values():
        if node.size > amt_to_free and (top_candidate is None or node.size < top_candidate.size):
            top_candidate = node
    print(top_candidate.val, top_candidate.size)
    return top_candidate.size
        

def pipeline2(lines):
    nodes = build_tree(lines)
    # for node in nodes.values():
    #     print(node.val, node.size)
    root = nodes[" /"]
    dfs(root)
    
    return conditioned_result_part2(nodes)


print(pipeline2(lines))

 / jsv bwc jwrhclh 10475598
10475598


## Messy solution (from being tired and rushed) dont look 🙈

In [5]:
# MESSY solution
# this is the result of a rushed attempt
from collections import defaultdict
import json

with open('input.txt') as f:
    lines = f.readlines()
tree = {}
current_path = []
hash_space = defaultdict()
paths_str = []
paths = []
inserts = defaultdict(set)

dirs = set()

for idx, line in enumerate(lines):
    if (line.startswith("$ cd")):
        if (line[5:-1] == ".."):
            # remove the first element
            current_path.pop()
        else:
            # add to the first position
            dirs.add(line[5:-1])
            current_path.append(line[5:-1])
            # current_path.insert(0, line[5:-1])
        # navigate the tre
        if (str(current_path[:]) not in paths_str):
            paths_str.append(str(current_path[:]))
            paths.append(current_path[:])

for path in paths:
    for idx, dir in enumerate(path):
        if (idx < len(path) - 1):
            inserts[dir].add(path[idx + 1])

sizes = defaultdict(int)
for dir in dirs:
    size = 0
    is_dir = False
    is_files = False
    for line in lines:
        # print(line)
        if (line.startswith("$ cd " + dir)):
            is_dir = True
        elif (line.startswith("$ ls")):
            is_files = True
        elif (not line.startswith("$") and is_dir and is_files):
            try:
                size += int(line.split(" ")[0])
            except:
                pass
        elif (line.startswith("$ cd ")):
            is_dir = False
            is_files = False
        elif (line.startswith("$") and is_dir and is_files):
            break
    sizes[dir] = size
# print(inserts)
# print(dirs)
# print(sizes)

def dfs(dir,navigated = []):
    navigated.append(dir)
    size = sizes[dir]
    if dir in inserts:
        for insert in inserts[dir]:
            if insert not in navigated:
                size += dfs(insert)
    return size
# print(dfs("jqmgn"))
# print(dirs)
sum_sizes = 0
for dir in list(dirs):
    t_size = dfs(dir)
    if (t_size < 100000):
        sum_sizes += t_size
print(sum_sizes)
    

    





1173873


In [6]:
def pipeline(lines):
    return lines
pipeline(lines)

['$ cd /\n',
 '$ ls\n',
 'dir a\n',
 '14848514 b.txt\n',
 '8504156 c.dat\n',
 'dir d\n',
 '$ cd a\n',
 '$ ls\n',
 'dir e\n',
 '29116 f\n',
 '2557 g\n',
 '62596 h.lst\n',
 '$ cd e\n',
 '$ ls\n',
 '584 i\n',
 '$ cd ..\n',
 '$ cd ..\n',
 '$ cd d\n',
 '$ ls\n',
 '4060174 j\n',
 '8033020 d.log\n',
 '5626152 d.ext\n',
 '7214296 k']